In [ ]:
from kafka import KafkaProducer
import datetime
import schedule
import requests
import json
import time

# Access token
token = '...'

# API
stock_symbols = 'AAPL,DIS,MSFT'
url = 'https://api.worldtradingdata.com/api/v1/stock?symbol=' + stock_symbols + '&api_token=' + token
producer = KafkaProducer(
    bootstrap_servers = ['localhost:9092'], 
    value_serializer = lambda x: json.dumps(x).encode('utf-8')
)

def get_stock_data(url, topic_name):
    try:
        # Request data
        response = requests.get(url)

        # Determine if response is appropriate
        response.raise_for_status()
    except requests.exceptions.HTTPError as error:
        print('Invalid response: {}'.format(error.response.status_code))
        sys.exit(1)
    
    content = json.loads(response.text)
    for i in range(0, len(content['data'])):
        symbol = content['data'][i]['symbol']
        company_name = content['data'][i]['name']
        price = content['data'][i]['price']
        day_change = content['data'][i]['day_change']
        
        # Create and send message
        send_message = {'Symbol': symbol, 'Company': company_name, 'Price': price, 'DayChange': day_change, 'Time': time.strftime('%m-%d-%Y %H:%M:%S')}
        producer.send(topic_name, send_message)
        
# Scheduler
def job():
    get_stock_data(url, 'stockdata')
    
schedule.every(1).minutes.do(job)

start = datetime.time(9,30)
close = datetime.time(22,0)

while True:
    if datetime.datetime.now().time() >= start and datetime.datetime.now().time() <= close:
        schedule.run_pending()
        time.sleep(1)